Thêm thư viện cần thiết


In [1]:
import pandas as pd
import numpy as np
import sys
import sklearn
import io
import random
import time
import pickle
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Đọc dữ liệu từ dataset
train_url = 'https://raw.githubusercontent.com/merteroglu/NSL-KDD-Network-Instrusion-Detection/master/NSL_KDD_Train.csv'
test_url = 'https://raw.githubusercontent.com/merteroglu/NSL-KDD-Network-Instrusion-Detection/master/NSL_KDD_Test.csv'

col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

df = pd.read_csv(train_url,header=None, names = col_names)

df_test = pd.read_csv(test_url, header=None, names = col_names)

print("Dimensions of Training set: ", df.shape)
print("Dimensions of Test set: ", df_test.shape)

Dimensions of Training set:  (125973, 42)
Dimensions of Test set:  (22544, 42)


In [3]:
# Xử lí dữ liệu trên 3 cột đặc biệt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
categorical_columns=['protocol_type', 'service', 'flag']

df_categorical_values = df[categorical_columns]
testdf_categorical_values = df_test[categorical_columns]
df_categorical_values.head()

# protocol type
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
print(unique_protocol2)

# service
unique_service=sorted(df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
print(unique_service2)


# flag
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
print(unique_flag2)


# put together
dumcols=unique_protocol2 + unique_service2 + unique_flag2


#do it for test set
unique_service_test=sorted(df_test.service.unique())
unique_service2_test=[string2 + x for x in unique_service_test]
testdumcols=unique_protocol2 + unique_service2_test + unique_flag2

print(dumcols)
print(testdumcols)

['Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp']
['service_IRC', 'service_X11', 'service_Z39_50', 'service_aol', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard', 'service_domain', 'service_domain_u', 'service_echo', 'service_eco_i', 'service_ecr_i', 'service_efs', 'service_exec', 'service_finger', 'service_ftp', 'service_ftp_data', 'service_gopher', 'service_harvest', 'service_hostnames', 'service_http', 'service_http_2784', 'service_http_443', 'service_http_8001', 'service_imap4', 'service_iso_tsap', 'service_klogin', 'service_kshell', 'service_ldap', 'service_link', 'service_login', 'service_mtp', 'service_name', 'service_netbios_dgm', 'service_netbios_ns', 'service_netbios_ssn', 'service_netstat', 'service_nnsp', 'service_nntp', 'service_ntp_u', 'service_other', 'service_pm_dump', 'service_pop_2', 'service_pop_3', 'service_printer', 'service_private', 'service_red_i', 'service_remote_job', 'ser

In [4]:
# convert data on 3 columns to number

# Train set
df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)

# test
print(df_categorical_values.head())
print('--------------------')
print(df_categorical_values_enc.head())

# Test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)

  protocol_type   service flag
0           tcp  ftp_data   SF
1           udp     other   SF
2           tcp   private   S0
3           tcp      http   SF
4           tcp      http   SF
--------------------
   protocol_type  service  flag
0              1       20     9
1              2       44     9
2              1       49     5
3              1       24     9
4              1       24     9


In [5]:
# One hot encoder
enc = OneHotEncoder(categories='auto')
df_categorical_values_encenc = enc.fit_transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc.toarray(),columns=dumcols)


# test set
testdf_categorical_values_encenc = enc.fit_transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc.toarray(),columns=testdumcols)

df_cat_data.head()

,Protocol_type_icmp,Protocol_type_tcp,Protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
trainservice=df['service'].tolist()
testservice= df_test['service'].tolist()
difference=list(set(trainservice) - set(testservice))
string = 'service_'
difference=[string + x for x in difference]

for col in difference:
    testdf_cat_data[col] = 0

print(df_cat_data.shape)
print(testdf_cat_data.shape)

(125973, 84)
(22544, 84)


In [7]:
newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)

# test data
newdf_test=df_test.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)

print(newdf.shape)
print(newdf_test.shape)
display(newdf)
display(newdf_test)

(125973, 123)
(22544, 123)


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,491,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,146,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0,232,8153,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,199,420,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
125969,8,105,145,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
125970,0,2231,384,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
125971,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_S2,flag_S3,flag_SF,flag_SH,service_aol,service_urh_i,service_harvest,service_red_i,service_http_8001,service_http_2784
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,2,12983,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
3,0,20,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
4,1,0,15,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,794,333,0,0,0,0,0,1,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
22540,0,317,938,0,0,0,0,0,1,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
22541,0,54540,8314,0,0,0,2,0,1,1,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
22542,0,42,42,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0


In [8]:
labeldf=newdf['label']
labeldf_test=newdf_test['label']


# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 1,'nmap' : 1,'portsweep' : 1,'satan' : 1,'mscan' : 1,'saint' : 1,
                            'ftp_write': 1,'guess_passwd': 1,'imap': 1,'multihop': 1,'phf': 1,'spy': 1,'warezclient': 1,'warezmaster': 1,'sendmail': 1,'named': 1,'snmpgetattack': 1,'snmpguess': 1,'xlock': 1,'xsnoop': 1,'httptunnel': 1,
                           'buffer_overflow': 1,'loadmodule': 1,'perl': 1,'rootkit': 1,'ps': 1,'sqlattack': 1,'xterm': 1 })
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 1,'nmap' : 1,'portsweep' : 1,'satan' : 1,'mscan' : 1,'saint' : 1
                           ,'ftp_write': 1,'guess_passwd': 1,'imap': 1,'multihop': 1,'phf': 1,'spy': 1,'warezclient': 1,'warezmaster': 1,'sendmail': 1,'named': 1,'snmpgetattack': 1,'snmpguess': 1,'xlock': 1,'xsnoop': 1,'httptunnel': 1,
                           'buffer_overflow': 1,'loadmodule': 1,'perl': 1,'rootkit': 1,'ps': 1,'sqlattack': 1,'xterm': 1})


# put the new label column back
newdf['label'] = newlabeldf
newdf_test['label'] = newlabeldf_test

In [9]:
# Split dataframes into X & Y
# X Özellikler , Y sonuç değişkenleri
X_Df = newdf.drop(labels='label',axis=1)
Y_Df = newdf.label

# test set
X_Df_test = newdf_test.drop(labels='label',axis=1)
Y_Df_test = newdf_test.label

In [10]:
colNames=list(X_Df)
colNames_test=list(X_Df_test)

In [11]:
print(colNames)
print(colNames_test)

['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp', 'service_IRC', 'service_X11', 'service_Z39_50', 'service_aol', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard', 'service_domain', 'service_domain_u', 'service_echo', 'se

In [12]:
#Chuẩn hóa dữ liệu
scaler_minmax = MinMaxScaler(feature_range=(0, 1))
scaler_minmax.fit(X_Df_test)

# with open('/content/drive/MyDrive/Model/scaler.pkl', 'rb') as file:
#     scaler_minmax = pickle.load(file)

MinMaxScaler()

In [13]:
print(scaler_minmax)

MinMaxScaler()


Load Model

In [14]:
# Voting classifier
voting_classifier = pickle.load(open("/content/drive/MyDrive/Model/voting_modelv1.pickle", "rb"))

In [15]:
# KNN model
knn_model = pickle.load(open("/content/drive/MyDrive/Model/knn_modelv2.pickle", "rb"))

In [16]:
# SVM model
svm_model = pickle.load(open("/content/drive/MyDrive/Model/svm_modelv1.pickle", "rb"))


In [17]:
# NB model
nb_model = pickle.load(open("/content/drive/MyDrive/Model/nb_modelv1.pickle", "rb"))


In [18]:
# DT model
dt_model = pickle.load(open("/content/drive/MyDrive/Model/dt_modelv1.pickle", "rb"))


In [19]:
# LR model
lr_model = pickle.load(open("/content/drive/MyDrive/Model/lr_modelv1.pickle", "rb"))

In [20]:
# RF model
rf_model = pickle.load(open("/content/drive/MyDrive/Model/rf_modelv1.pickle", "rb"))

In [21]:
# QDA model
qda_model = pickle.load(open("/content/drive/MyDrive/Model/qda_modelv1.pickle", "rb"))

In [22]:
# MLP model
mlp_model = pickle.load(open("/content/drive/MyDrive/Model/mlp_modelv1.pickle", "rb"))

In [23]:
# LDA model
lda_model = pickle.load(open("/content/drive/MyDrive/Model/lda_modelv1.pickle", "rb"))

In [24]:
# GB model
gb_model = pickle.load(open("/content/drive/MyDrive/Model/gb_modelv1.pickle", "rb"))

In [25]:
# BAG model
bag_model = pickle.load(open("/content/drive/MyDrive/Model/bag_modelv1.pickle", "rb"))

In [45]:
# Spilit all vector malicious out Dataset
malicious_vector = newdf_test.loc[newdf_test['label'] == 1]
malicious_vector.reset_index(drop=True, inplace=True)
malicious_vector.drop('label', axis=1, inplace=True)
malicious_vector.head()

<ipython-input-45-3ba5973c6e51>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  malicious_vector.drop('label', axis=1, inplace=True)


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_S2,flag_S3,flag_SF,flag_SH,service_aol,service_urh_i,service_harvest,service_red_i,service_http_8001,service_http_2784
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,0,20,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
3,1,0,15,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,0,129,174,0,0,0,0,1,0,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0


In [27]:
# Function Evaluate
def Evaluate(proba_list):
  sum = 0
  for proba in proba_list:
    sum += proba[0]
  result = sum / len(proba_list)
  return result

In [46]:
original_vector = malicious_vector.drop(malicious_vector.index[:1000])
original_data = scaler_minmax.transform(original_vector)

In [47]:
# Evaluate on models

# SVM model
svm_proba = svm_model.predict_proba(original_data)
result_svm = Evaluate(svm_proba)
print(f"original vector | SVM model: {result_svm}")

# KNN model
knn_proba = knn_model.predict_proba(original_data)
result_knn = Evaluate(knn_proba)
print(f"original vector | KNN model: {result_knn}")

# DT model
dt_proba = dt_model.predict_proba(original_data)
result_dt = Evaluate(dt_proba)
print(f"original vector | DT model: {result_dt}")

# NB model
nb_proba = nb_model.predict_proba(original_data)
result_nb = Evaluate(nb_proba)
print(f"original vector | NB model: {result_nb}")

# LR model
lr_proba = lr_model.predict_proba(original_data)
result_lr = Evaluate(lr_proba)
print(f"original vector | LR model: {result_lr}")

# RF model
rf_proba = rf_model.predict_proba(original_data)
result_rf = Evaluate(rf_proba)
print(f"original vector | RF model: {result_rf}")

# GB model
gb_proba = gb_model.predict_proba(original_data)
result_gb = Evaluate(gb_proba)
print(f"original vector | GB model: {result_gb}")

# MLP model
mlp_proba = mlp_model.predict_proba(original_data)
result_mlp = Evaluate(mlp_proba)
print(f"original vector | MLP model: {result_mlp}")

# QDA model
qda_proba = qda_model.predict_proba(original_data)
result_qda = Evaluate(qda_proba)
print(f"original vector | QDA model: {result_qda}")

# LDA model
lda_proba = lda_model.predict_proba(original_data)
result_lda = Evaluate(lda_proba)
print(f"original vector | LDA model: {result_lda}")

# BAG model
bag_proba = bag_model.predict_proba(original_data)
result_bag = Evaluate(bag_proba)
print(f"original vector | BAG model: {result_bag}")


original vector | SVM model: 0.31542395835304504
original vector | KNN model: 0.3803205724104894
original vector | DT model: 0.44644211049103205
original vector | NB model: 0.10402875934038461
original vector | LR model: 0.2507928779353984
original vector | RF model: 0.44644211049103205
original vector | GB model: 0.37364514574521257
original vector | MLP model: 0.10733871410053784
original vector | QDA model: 0.006422716132848813
original vector | LDA model: 0.3608619040421142
original vector | BAG model: 0.3592925152821291


GENETIC ALGORITHM

In [39]:
# Genetic Algorithm

# define range value of features
tags = ["duration","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

cols = tags + dumcols

param_range = {
    'duration':[0,8445],
    'src_bytes':[0,1300000],
    'dst_bytes':[0, 1300000],
    'land':[0,1],
    'wrong_fragment': [0, 10],
    'urgent': [0, 10],
    'hot': [0,101],
    'num_failed_logins':[0, 10],
    'logged_in': [0,1],
    'num_compromised': [0,7479],
    'root_shell': [0,1],
    'su_attempted': [0, 10],
    'num_root': [0,7468],
    'num_file_creations': [0, 10],
    'num_shells':[0, 10],
    'num_access_files':[0, 10],
    'num_outbound_cmds': [0,10],
    'is_host_login':[0,1],
    'is_guest_login':[0,1],
    'count':[0,511],
    'srv_count':[0,511],
    'serror_rate':[0,1],
    'srv_serror_rate':[0,1],
    'rerror_rate':[0,1],
    'srv_rerror_rate':[0,1],
    'same_srv_rate':[0,1],
    'diff_srv_rate':[0,1],
    'srv_diff_host_rate':[0,1],
    'dst_host_count':[0,255],
    'dst_host_srv_count':[0,255],
    'dst_host_same_srv_rate':[0,1],
    'dst_host_diff_srv_rate':[0,1],
    'dst_host_same_src_port_rate':[0,1],
    'dst_host_srv_diff_host_rate':[0,1],
    'dst_host_serror_rate':[0,1],
    'dst_host_srv_serror_rate':[0,1],
    'dst_host_rerror_rate':[0,1],
    'dst_host_srv_rerror_rate':[0,1]
}

features_integer = ["duration","src_bytes",
    "dst_bytes","wrong_fragment","urgent","hot","num_failed_logins",
    "num_compromised","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "count","srv_count"]

features_binary = ["land", "logged_in", "root_shell", "is_host_login", "is_guest_login"]

index_integer = [0, 1, 2, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 19, 20, 28, 29]
index_binary = [3, 8, 10, 18]

# Creae randomly a vector
def generate_random_vector(original):
    vector = []
    i = 0
    for tag in tags:
        if tag in features_integer:
            # if tag == 'duration':
            #   vector.append(original[0])
            #   continue
            vector.append(random.randint(original[i], param_range[tag][1]))
        elif tag in features_binary:
            vector.append(random.choice([0,1]))
        else:
            vector.append(round(random.uniform(0, 1),2))
        i += 1
    return vector

# Init population of genetic algorithm
def init_population(original_vector, population_size):
    population = [original_vector]
    for _ in range(population_size - 1):
        vector = generate_random_vector(original_vector)
        population.append(vector)
    return population

# Selection
def roulete_wheel(fitness_score):
    total = sum(score for score in fitness_score)

    spin = random.uniform(0, total)
    current_sum = 0
    for i in range(fitness_score):
        current_sum += fitness_score[i]
        if current_sum >= spin:
            return i

# Select parent operate
def select_parent(population):
    population_size = len(population)
    i = random.randint(0, population_size - 1)
    j = random.randint(0, population_size - 1)
    while i == j:
        j = random.randint(0, population_size - 1)

    parent = [population[i], population[j]]
    return parent

# Crossover operate
def crossover(population, population_size, crossover_rate):
    number = int(population_size * crossover_rate)

    for _ in range(number):
        parent = select_parent(population)
        index_crossover = random.randint(0, 37)
        offspring1 = parent[0][:index_crossover] + parent[1][index_crossover:]
        offspring2 = parent[1][:index_crossover] + parent[0][index_crossover:]
        population.append(offspring1)
        population.append(offspring2)

# Mutate operate
def mutate(population, population_size, mutation_rate):
    for i in range(len(population)):
        number = random.uniform(0,1)
        if(number <= mutation_rate):
            index_mutate = random.randint(0,37)
            if index_mutate in index_integer:
                # if index_mutate == 0:
                #   continue
                p = param_range[tags[index_mutate]]
                population[i][index_mutate] = random.randint(int(population[i][index_mutate]), p[1])
            elif index_mutate in index_binary:
                if population[i][index_mutate] == 1:
                  population[i][index_mutate] = 0
                else:
                  population[i][index_mutate] = 1
            else:
                population[i][index_mutate] = round(random.uniform(population[i][index_mutate],1),2)

# Calculate fitness
def fitness_function(population, immutable):
    p = population[:]
    for i in range(len(p)):
        p[i] = p[i] + immutable

    df_population = pd.DataFrame(p, columns=colNames_test)
    df_population[features_integer] = df_population[features_integer].astype(int)
    df_population[features_binary] = df_population[features_binary].astype(int)

    #s = preprocessing.StandardScaler().fit(df_population)
    #scaler_minmax = MinMaxScaler(feature_range=(0, 1))
    #scaler_minmax.fit(df_population)

    test=scaler_minmax.transform(p)
    result = voting_classifier.predict_proba(test)
    fitness_score = [proba[0] for proba in result]
    return fitness_score

def sumfit(ls):
  sum = 0
  for value in ls:
    sum+=value
  return value

# Update population for next generation
def update_population(population, population_size, fitness_score):
    new_population = []
    scores = fitness_score[:]

    for _ in range(population_size):
        total_score = sumfit(scores)
        spin = random.uniform(0,total_score)
        current_sum = 0
        for i in range(len(scores)):
            current_sum += scores[i]
            if current_sum >= spin:
                scores.pop(i)
                element = population.pop(i)
                new_population.append(element)
                break
    return new_population

def genetic_algorithm(original_vector, population_size, generations, crossover_rate, mutation_rate, improvement_mini):
  # original_vector is a Series

  # Split original_vector to immutable and mutable section
    immutable = original_vector[dumcols].tolist()
    mutable = original_vector.drop(labels=dumcols).tolist()

    # List all invidual best fitness
    subsequent = []
    bestscore = []

    # Init population base on mutable vector
    population = init_population(mutable, population_size)

    # Init average fitness of population
    fitness_score = fitness_function(population, immutable)
    total = sumfit(fitness_score)
    avg_fitness = total / len(fitness_score)
    print(f"Average fitness of population before is: {avg_fitness}")

    # Init best fitness of population is 0
    best_fitness = 0
    print(f"Best fitness of population before is: {best_fitness}")

    #init iteration
    iteration = 1

    # init improvement ratio
    improvement = 0
    count = 0

    # init a seed for random
    random.seed(int(time.time()))
    while iteration <= generations and (improvement >= improvement_mini or count <= 5):
        # cross-over
        crossover(population, population_size, crossover_rate)

        # mutate
        mutate(population, population_size, mutation_rate)

        # calculate fitness functions
        fitness_score = fitness_function(population, immutable)

        # calculate average fitness of population
        total_score = sumfit(fitness_score)
        avg_fitness_new = total_score / len(fitness_score)

        # update population best fitness
        max = 0
        index = 0
        for i in range(len(fitness_score)):
            if fitness_score[i] > max:
                max = fitness_score[i]
                index = i
        best_fitness_new = max
        bestscore.append(max)

        # calculate improvement rate
        improvement = (avg_fitness_new - avg_fitness) / avg_fitness
        improvement *= 100
        avg_fitness = avg_fitness_new
        if improvement < improvement_mini:
            count+=1
        #else:
            #count = 0

        # save invidual best fitness in subsequen list
        subsequent.append(population[index])

        # update population
        population.pop(index)
        fitness_score.pop(index)

        # update population for next generation
        population = update_population(population, population_size, fitness_score)

        if iteration % 10 == 0:
          print(f"Best fitness of population is: {best_fitness_new}")
          print(f"Generation {iteration} average fitness is: {avg_fitness_new}")
          print(f"improvement is: {improvement}")

        iteration+=1
    print("Completed!")

    max = bestscore[0]
    index_bestscore = 0
    for i in range(len(bestscore)):
      if bestscore[i] > max:
        max = bestscore[i]
        index_bestscore = i
    #for i in range(len(subsequent)):
    subsequent[index_bestscore] = subsequent[index_bestscore] + immutable
    print(max)
    return subsequent[index_bestscore]


In [192]:

listtest = []
for i in range(5):
  print(f"--------------row {i}:-------------")
  original_vector = malicious_vector.loc[i]
  result_adv_ga = genetic_algorithm(original_vector, 100, 1000, 0.25, 0.2, 0.01)
  listtest.append(result_adv_ga)

test_adv_ga = scaler_minmax.transform(listtest)

proba_adv_ga = svm_model.predict_proba(test_adv_ga)

sum = 0
for pr in proba_adv_ga:
  sum += pr[0]
print(sum / len(proba_adv_ga))


Output hidden; open in https://colab.research.google.com to view.

In [31]:
adv_ga_df = pd.DataFrame(listtest, columns=colNames_test)
adv_ga_df[features_integer] = adv_ga_df[features_integer].astype(int)
adv_ga_df[features_binary] = adv_ga_df[features_binary].astype(int)

adv_ga_df.to_csv('/content/drive/MyDrive/Model/adv_ga.csv', index=False)

NameError: ignored

In [38]:
print(adv_ga_df.shape)

(1000, 122)


In [67]:
proba_adv_ga = dt_model.predict_proba(test_adv_ga)
print(Evaluate(proba_adv_ga))
print(proba_adv_ga)

proba_adv_ga_test = dt_model.predict(test_adv_ga)
print(proba_adv_ga_test)


0.8909922570334242
[[0.89099226 0.10900774]
 [0.89099226 0.10900774]]
[0 0]


In [181]:

listtest = []
for i in range(20):
  original_vector = malicious_vector.loc[i]
  result_adv_ga = genetic_algorithm(original_vector, 100, 1000, 0.25, 0.2, 0.01)
  listtest.append(result_adv_ga)

  #scaler_test = preprocessing.StandardScaler().fit(result_adv_ga)
test_adv_ga = scaler_minmax.transform(listtest)

proba_adv_ga = svm_model.predict_proba(test_adv_ga)

sum = 0
for pr in proba_adv_ga:
  sum += pr[0]
print(sum / len(proba_adv_ga))


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Average fitness of population before is: 0.0025977593130305453
Best fitness of population before is: 0


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScal

Best fitness of population is: 0.9727479440235286
Generation 10 average fitness is: 0.005373635825573449
improvement is: 6.089884702381136


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScal

Best fitness of population is: 0.9727480392583535
Generation 10 average fitness is: 0.004818320451852168
improvement is: 41.77759985023102


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Completed!
0.9727480392583535
0.9999998986268693


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [191]:
proba_adv_ga =  mlp_model.predict_proba(test_adv_ga)

sum = 0
for pr in proba_adv_ga:
  sum += pr[0]
print(sum / len(proba_adv_ga))

0.775660558052224


In [140]:
print(proba_adv_ga)

[[1.00000000e+00 2.71471402e-13]
 [9.99963991e-01 3.60085073e-05]
 [1.00000000e+00 1.54223221e-20]
 [1.00000000e+00 3.35954448e-60]
 [1.00000000e+00 1.11751529e-31]]


In [48]:
adv_url = '/content/drive/MyDrive/Model/adv_ga.csv'


In [49]:
df_test_adv_ga = pd.read_csv(adv_url, header=None, names = colNames_test)
df_test_adv_ga.drop(0)

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_S2,flag_S3,flag_SF,flag_SH,service_aol,service_urh_i,service_harvest,service_red_i,service_http_8001,service_http_2784
1,6948,11937,950952,1,0,10,3,0,1,1618,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2502,1169064,281996,0,2,0,4,1,0,1909,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4342,1220921,1081598,1,2,1,1,0,1,314,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,7792,353754,801610,0,0,1,20,9,0,3560,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,3537,280031,263903,1,1,0,10,9,1,757,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,8021,939226,263998,0,1,0,15,1,1,1015,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
997,309,650100,1051274,0,0,7,6,4,1,6730,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,2279,575119,242078,0,5,5,8,6,1,6848,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
999,6114,1074726,1050040,0,1,7,4,3,1,930,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [50]:
f = df_test_adv_ga.drop(0)


In [51]:
data_adv = scaler_minmax.transform(f)

In [52]:
adv_svm = svm_model.predict_proba(data_adv)
print(f"ADV GA on SVM MODEL: {Evaluate(adv_svm)}")

adv_dt = dt_model.predict_proba(data_adv)
print(f"ADV GA on DT MODEL: {Evaluate(adv_dt)}")

adv_nb = nb_model.predict_proba(data_adv)
print(f"ADV GA on NB MODEL: {Evaluate(adv_nb)}")

adv_knn = knn_model.predict_proba(data_adv)
print(f"ADV GA on KNN MODEL: {Evaluate(adv_knn)}")

adv_rf = rf_model.predict_proba(data_adv)
print(f"ADV GA on RF MODEL: {Evaluate(adv_rf)}")

adv_mlp = mlp_model.predict_proba(data_adv)
print(f"ADV MLP on MLP MODEL: {Evaluate(adv_mlp)}")

adv_gb = gb_model.predict_proba(data_adv)
print(f"ADV GA on GB MODEL: {Evaluate(adv_gb)}")

adv_lr = lr_model.predict_proba(data_adv)
print(f"ADV GA on lr MODEL: {Evaluate(adv_lr)}")

adv_lda = lda_model.predict_proba(data_adv)
print(f"ADV GA on LDA MODEL: {Evaluate(adv_lda)}")

adv_qda = qda_model.predict_proba(data_adv)
print(f"ADV GA on QDA MODEL: {Evaluate(adv_qda)}")

adv_bag = bag_model.predict_proba(data_adv)
print(f"ADV GA on BAG MODEL: {Evaluate(adv_bag)}")


ADV GA on SVM MODEL: 0.9906548328322589
ADV GA on DT MODEL: 0.8209395039130365
ADV GA on NB MODEL: 0.965
ADV GA on KNN MODEL: 0.9453333333333332
ADV GA on RF MODEL: 0.8209395039130365
ADV MLP on MLP MODEL: 0.7154016556010391
ADV GA on GB MODEL: 0.08072121461611664
ADV GA on lr MODEL: 0.30616420091447255
ADV GA on LDA MODEL: 0.5005792318398194
ADV GA on QDA MODEL: 0.091
ADV GA on BAG MODEL: 0.49233333333333296


In [54]:
print((Evaluate(adv_svm) + Evaluate(adv_dt) + Evaluate(adv_nb) + Evaluate(adv_knn) + Evaluate(adv_rf) + Evaluate(adv_mlp) + Evaluate(adv_gb) + Evaluate(adv_lr) + Evaluate(adv_lda) + Evaluate(adv_qda) + Evaluate(adv_bag)) / 11)

0.611733346390586
